In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint

In [2]:
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []
elec_int = []
resin_scenario_l = []
ix_cap_list = []
ix_onm_list = []
ro_cap_list = []
ro_onm_list = []

def save_results(scenario = None, s_value = None, resin_name = None):
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(s_value)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    elec_int.append(value(m.fs.costing.electricity_intensity))
    if resin_name is None:
        resin_scenario_l.append(0)
        ix_cap_list.append(0)
        ix_onm_list.append(0)    
    else:
        resin_scenario_l.append(resin_name)
        ix_cap_list.append(m.fs.ion_exchange.costing.fixed_cap_inv())
        ix_onm_list.append(m.fs.ion_exchange.costing.annual_op_main_cost())
    
    ro_cap = 0
    ro_onm = 0
    for key in m.fs.pfd_dict.keys():
        if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
            ro_cap = getattr(m.fs, key).costing.fixed_cap_inv() + ro_cap
            ro_onm = getattr(m.fs, key).costing.annual_op_main_cost() + ro_onm
    ro_cap_list.append(ro_cap)
    ro_onm_list.append(ro_onm)
        

In [3]:
wt.case_study_trains.source_water = {"case_study": "kbhdp", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "kbhdp_brackish_ave"}

In [4]:
m = wt.watertap_setup(dynamic=False)

wt.case_study_trains.train = {"case_study": "kbhdp",
                             "reference": "nawi",
                             "scenario": "ps_foa_baseline"}

m = wt.case_study_trains.get_case_study(m=m)

### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, skip_small=True)

m = wt.set_bounds(m)

wt.print_ro_results(m)

# SAVE RESULTS.
df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario="ps_foa_baseline")

save_results(scenario = "ps_foa_baseline", s_value = "n/a")

kbhdp

------- Adding Unit Processes -------
well_field
media_filtration
anti_scalant_addition
ro_first_pass
ro_first_stage
caustic_soda_addition
chlorination
treated_storage
deep_well_injection
municipal_drinking
surface_discharge
-------------------------------------

adding source
adding splitter
params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0

In [5]:
m = wt.watertap_setup(dynamic=False)

wt.case_study_trains.train = {"case_study": "kbhdp",
                             "reference": "nawi",
                             "scenario": "ps_foa"}

m = wt.case_study_trains.get_case_study(m=m)

#m.fs.kbhdp_brackish_ave.flow_vol_in.fix(0.0043812636 / 0.87991)
#m.fs.kbhdp_brackish_ave.conc_mass_in[0, "tds"].fix(8.561)
#m.fs.anion_exchange.water_recovery.fix(1 - (12/100))

#m.fs.ion_exchange.removal_fraction[0, "tds"].fix(0.95)

### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, skip_small=True)

m.fs.ion_exchange.removal_fraction[0, "tds"].fix(0.95)

m = wt.set_bounds(m)

wt.print_ro_results(m)


kbhdp

------- Adding Unit Processes -------
well_field
media_filtration
ion_exchange
anti_scalant_addition
ro_first_pass
ro_first_stage
caustic_soda_addition
chlorination
treated_storage
deep_well_injection
municipal_drinking
surface_discharge
-------------------------------------

adding source
adding splitter
params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electric

In [6]:
# SAVE RESULTS.
df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario="ps_foa_ix_95")

#save_results(scenario = "ix_tds_recovery_base", s_value = 0.95, resin_name = m.fs.ion_exchange.resin_name)

In [7]:
m = wt.watertap_setup(dynamic=False)

wt.case_study_trains.train = {"case_study": "kbhdp",
                             "reference": "nawi",
                             "scenario": "ps_foa"}

m = wt.case_study_trains.get_case_study(m=m)

### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, skip_small=True)

m.fs.ion_exchange.removal_fraction[0, "tds"].fix(0.25)

m = wt.set_bounds(m)

wt.print_ro_results(m)

# SAVE RESULTS.
df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario="ps_foa_ix_25")

#save_results(scenario = "ix_tds_recovery_base", s_value = 0.25, resin_name = m.fs.ion_exchange.resin_name)


kbhdp

------- Adding Unit Processes -------
well_field
media_filtration
ion_exchange
anti_scalant_addition
ro_first_pass
ro_first_stage
caustic_soda_addition
chlorination
treated_storage
deep_well_injection
municipal_drinking
surface_discharge
-------------------------------------

adding source
adding splitter
params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electric

In [8]:
m = wt.watertap_setup(dynamic=False)

wt.case_study_trains.train = {"case_study": "kbhdp",
                             "reference": "nawi",
                             "scenario": "ps_foa"}

m = wt.case_study_trains.get_case_study(m=m)

### # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("Unfixing feed presure and area for", key, '...\n')

wt.run_water_tap(m=m, objective=True, skip_small=True)

m.fs.ion_exchange.removal_fraction[0, "tds"].fix(0.5)

m = wt.set_bounds(m)

wt.print_ro_results(m)

# SAVE RESULTS.
df = wt.get_results_table(m=m, case_study=wt.case_study_trains.source_water["case_study"], 
                                scenario="ps_foa_ix_50")

save_results(scenario = "ix_foa_baseline", s_value = 0.8, resin_name = m.fs.ion_exchange.resin_name)

kbhdp

------- Adding Unit Processes -------
well_field
media_filtration
ion_exchange
anti_scalant_addition
ro_first_pass
ro_first_stage
caustic_soda_addition
chlorination
treated_storage
deep_well_injection
municipal_drinking
surface_discharge
-------------------------------------

adding source
adding splitter
params into splitter --> [0.393, 0.607]

Connecting unit processes...
Unfixing feed presure and area for ro_first_pass ...

Unfixing feed presure and area for ro_first_stage ...

----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------
----------------------------------------------------------------------

Degrees of Freedom: 8

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electric

In [9]:
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.fix(getattr(m.fs, key).feed.pressure[0]())
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())
        getattr(m.fs, key).a.fix(getattr(m.fs, key).a[0]())
        getattr(m.fs, key).b.fix(getattr(m.fs, key).b[0]())     

In [10]:
wt.run_water_tap(m=m, objective=True, print_model_results="summary", skip_small=True)

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 5.46165
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.07273

ION EXCHANGE:

	total cap investment: 10.18798
	cat and chem cost: 0.28376
	electricity cost: 0.06271
	total fixed op cost: 0.1358

ANTI SCALANT ADDITION:

	total cap investment: 0.09389
	cat and chem cost: 0.4118
	electricity cost: 0.0
	total fixed op cost: 0.00125

RO FIRST PASS:

	total cap investment: 6.65299
	cat and chem cost: 0.06327
	electricity cost: 0.43089
	total fixed op cost: 0.08859

RO FIRST STAGE:

	total cap investment: 3.60893
	cat and chem cost: 0.03432
	electricity cost: 0.062
	total fixed o

In [11]:
m = wt.set_bounds(m)

----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------

***UNIT PROCESS RESULTS (in $MM)***


WELL FIELD:

	total cap investment: 12.10508
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.1606

MEDIA FILTRATION:

	total cap investment: 5.46165
	cat and chem cost: 0.0
	electricity cost: 0.0
	total fixed op cost: 0.07273

ION EXCHANGE:

	total cap investment: 10.18798
	cat and chem cost: 0.28376
	electricity cost: 0.06271
	total fixed op cost: 0.1358

ANTI SCALANT ADDITION:

	total cap investment: 0.09389
	cat and chem cost: 0.4118
	electricity cost: 0.0
	total fixed op cost: 0.00125

RO FIRST PASS:

	total cap investment: 6.65299
	cat and chem cost: 0.06327
	electricity cost: 0.43089
	total fixed op cost: 0.08859

RO FIRST STAGE:

	total cap investment: 3.60893
	cat and chem cost: 0.03432
	electricity cost: 0.062
	total fixed o

In [12]:
resin_type_list = ['styrenic_gel_2']

In [13]:
runs_per_scenario = 20
ix = m.fs.ion_exchange

for resin_name in resin_type_list:
    ix.resin_cost.fix(ix.resin_dict[resin_name])
    print("-------", resin_name, "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())

    ############ onstream_factor 70-100% ############
    stash_value = m.fs.costing_param.plant_cap_utilization
    scenario = "Plant Capacity Utilization 70-100%"
    print("-------", scenario, "-------")
    ub = 1
    lb = 0.7
    step = (ub - lb) / runs_per_scenario
    
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.plant_cap_utilization = i
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())
        
        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    m.fs.costing_param.plant_cap_utilization = stash_value  
    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ lifetime years ############

    stash_value = value(m.fs.costing_param.plant_lifetime_yrs)
    scenario = "Plant Lifetime 15-45 yrs"
    print("-------", scenario, "-------")
    ub = 45
    lb = 15
    step = (ub - lb) / runs_per_scenario
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.plant_lifetime_yrs = i 
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    m.fs.costing_param.plant_lifetime_yrs = stash_value
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ elec cost +-30% ############

    stash_value = value(m.fs.costing_param.electricity_price)
    scenario = "electricity price +- 30%"
    print("-------", scenario, "-------")
    ub = stash_value * 1.3
    lb = stash_value * 0.7
    step = (ub - lb) / runs_per_scenario
    for i in np.arange(lb, ub + step, step):
        m.fs.costing_param.electricity_price = i 
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    m.fs.costing_param.electricity_price = stash_value

    
    ###########################################################    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())

    ############ esin price +- 40% ############

    stash_value = value(ix.resin_cost[0]())
    scenario = "resin price +- 50%"
    print("-------", scenario, "-------")
    ub = stash_value * 1.5
    lb = stash_value * 0.5
    step = (ub - lb) / runs_per_scenario
    for i in np.arange(lb, ub + step, step):
        ix.resin_cost.fix(i) 
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    ix.resin_cost.fix(stash_value)

    ###########################################################    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    
    
    ############ tds_removal 10-50% ############
    stash_value = ix.removal_fraction[0, "tds"]()
    scenario = "tds_removal 10-90%"
    print("-------", scenario, "-------")
    ub = 0.9
    lb = 0.1
    step = (ub - lb) / runs_per_scenario

    for key in m.fs.pfd_dict.keys():
        if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
            getattr(m.fs, key).feed.pressure.unfix()
            getattr(m.fs, key).membrane_area.unfix()

    for i in np.arange(lb, ub + step, step):
        ix.removal_fraction[0, "tds"].fix(i)   
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)

    ix.removal_fraction[0, "tds"].fix(stash_value)    
    ############################################################  
    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ inlet flow +-30% ############
    stash_value = []
    for key in m.fs.flow_in_dict:   
        stash_value.append(value(getattr(m.fs, key).flow_vol_in[0]))
    scenario = "Inlet Flow +80%"
    print("-------", scenario, "-------")
    ub = 1.8
    lb = 1.0
    step = (ub - lb) / runs_per_scenario

    for i in np.arange(lb, ub + step, step):
        q=0
        sensitivity_flow = {}
        for key in m.fs.flow_in_dict:
            getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q] * i)
            sensitivity_flow[key] = stash_value[q] * i
            q = q + 1

        wt.run_water_tap(m = m, objective=False, skip_small = True, sensitivity_flow = sensitivity_flow)
        print(scenario, stash_value[q-1] * i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    q=0
    for key in m.fs.flow_in_dict:    
        getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q])
        q = q + 1  
        
    ############################################################
    
    ############################################################
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())
    ############ inlet flow +-30% ############
    stash_value = []
    for key in m.fs.flow_in_dict:   
        stash_value.append(value(getattr(m.fs, key).conc_mass_in[0, "tds"]))
    scenario = "TDS inlet +50%"
    print("-------", scenario, "-------")
    ub = 2.0
    lb = 1.0
    step = (ub - lb) / runs_per_scenario

    for i in np.arange(lb, ub + step, step):
        q=0
        sensitivity_flow = {}
        for key in m.fs.flow_in_dict:
            getattr(m.fs, key).conc_mass_in[0, "tds"].fix(stash_value[q] * i)
            q = q + 1

        wt.run_water_tap(m = m, objective=False, skip_small = True, sensitivity_flow = sensitivity_flow)
        print(scenario, stash_value[q-1] * i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
        
    q=0
    for key in m.fs.flow_in_dict:    
        getattr(m.fs, key).conc_mass_in[0, "tds"].fix(stash_value[q])
        q = q + 1  
        
    ############################################################    
    
    
    print("-------", "RESET", "-------")
    wt.run_water_tap(m = m, objective=False, skip_small = True)
    print("LCOW -->", m.fs.costing.LCOW())        
        
    ############ capacity 10-50% ############
    
    stash_value1 = ix.anion_res_capacity[0]()
    stash_value2 = ix.cation_res_capacity[0]()
    
    scenario = "resin capacity 90 - 200%"
    print("-------", scenario, "-------")
    ub = 2
    lb = 0.9
    step = (ub - lb) / runs_per_scenario

    for i in np.arange(lb, ub + step, step):       
        ix.anion_res_capacity.fix(stash_value1 * i)
        ix.cation_res_capacity.fix(stash_value2 * i)
        
        wt.run_water_tap(m = m, objective=False, skip_small = True)
        print(scenario, i, "LCOW -->", m.fs.costing.LCOW())

        save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)

    ix.anion_res_capacity.fix(stash_value1)
    ix.cation_res_capacity.fix(stash_value2)
    
    ############################################################  
    
    
    

------- styrenic_gel_2 -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3010934586754339
------- Plant Capacity Utilization 70-100% -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.7 LCOW --> 0.430133512393477
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Capacity Utilization 70-100% 0.715 LCOW --> 0.4211097324131942
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------


Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 27.0 LCOW --> 0.3096963311025452
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 28.5 LCOW --> 0.3051332660271034
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 30.0 LCOW --> 0.3010934586754339
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Plant Lifetime 15-45 yrs 31.5 LCOW --> 0.29750088828687793
----------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06757999999999995 LCOW --> 0.3097358609508922
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.06921499999999994 LCOW --> 0.3108161612353245
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
electricity price +- 30% 0.07084999999999994 LCOW --> 0.31189646151975675
------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 0

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3010934586754339
------- resin price +-


WaterTAP3 solution optimal 

----------------------------------------------------------------------
tds_removal 10-90% 0.3400000000000001 LCOW --> 0.2907771721784606
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
tds_removal 10-90% 0.38 LCOW --> 0.293081014252719
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
tds_removal 10-90% 0.42000000000000004 LCOW --> 0.2955720163878486
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
tds_removal 10-90% 0.4600000000000001 LCOW --> 0.29824471595492186
---------------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +80% 2.191495920000001 LCOW --> 0.3274026079639815
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +80% 2.244947040000001 LCOW --> 0.3286469608905458
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +80% 2.298398160000001 LCOW --> 0.3298641200615021
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
Inlet Flow +80% 2.351849280000001 LCOW --> 0.3310562673425185
------------------------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
resin capacity 90 - 200% 1.12 LCOW --> 0.2951851131233178
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
resin capacity 90 - 200% 1.1750000000000003 LCOW --> 0.2929947042706961
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
resin capacity 90 - 200% 1.2300000000000004 LCOW --> 0.29105632452762537
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
resin capacity 90 - 200% 1.2850000000000004 LCOW --> 0.28933089535537604
--------------------------------------

In [14]:
sens_df = pd.DataFrame()

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["elec_int"] =  elec_int
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["resin"] = resin_scenario_l
sens_df["ix_cap"] = ix_cap_list
sens_df["ix_onm"] = ix_onm_list
sens_df["ro_cap"] = ro_cap_list
sens_df["ro_onm"] = ro_onm_list

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100

m_scenario = "ps_foa"
sens_df.to_csv("results/case_studies/%s_%s_sensitivity.csv" % ("KB_FOA", m_scenario))

In [15]:
sens_df

,lcow,water_recovery,elec_lcow,elec_int,scenario_value,scenario_name,lcow_difference,water_recovery_difference,resin,ix_cap,ix_onm,ro_cap,ro_onm
0,0.276844,82.267286,13.058117,0.663314,n/a,ps_foa_baseline,-0.000327,-0.029300,0,0.000000,0.000000,9.856476,1.341847
1,0.301093,85.141965,11.959745,0.660734,0.8,ix_foa_baseline,0.023923,-0.000553,styrenic_gel_2,9.688996,0.838894,9.759316,1.327134
2,0.430134,85.141965,11.959745,0.660734,0.7,Plant Capacity Utilization 70-100%,0.152963,-0.000553,styrenic_gel_2,9.688996,0.838894,9.759316,1.327134
3,0.421110,85.141965,11.959745,0.660734,0.715,Plant Capacity Utilization 70-100%,0.143939,-0.000553,styrenic_gel_2,9.688996,0.838894,9.759316,1.327134
4,0.412457,85.141965,11.959745,0.660734,0.73,Plant Capacity Utilization 70-100%,0.135286,-0.000553,styrenic_gel_2,9.688996,0.838894,9.759316,1.327134
...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,0.279619,85.190506,12.931169,0.663448,1.78,resin capacity 90 - 200%,0.002448,-0.000068,styrenic_gel_2,4.299402,0.413219,9.791020,1.331935
166,0.278937,85.192362,12.964818,0.663554,1.835,resin capacity 90 - 200%,0.001767,-0.000050,styrenic_gel_2,4.127198,0.399618,9.792249,1.332121
167,0.278305,85.194109,12.996200,0.663653,1.89,resin capacity 90 - 200%,0.001135,-0.000032,styrenic_gel_2,3.967418,0.386999,9.793406,1.332296
168,0.277718,85.195757,13.025526,0.663747,1.945,resin capacity 90 - 200%,0.000547,-0.000016,styrenic_gel_2,3.818815,0.375262,9.794499,1.332462


In [16]:
import pyomo.util.infeasible as infeas
print(infeas.log_infeasible_bounds(m))
print(infeas.log_infeasible_constraints(m))

None
INFO: CONSTR fs.ion_exchange.an_res_vol_constr: -24349.736537972538 =/= 0.0
INFO: CONSTR fs.ion_exchange.cation_res_vol_constr: -50289.53457722702 =/= 0.0
None


In [17]:
runs_per_scenario = 50
ix = m.fs.ion_exchange

In [18]:
sens_df = pd.DataFrame()

lcow_list = []
water_recovery_list = []
scenario_value = []
scenario_name = []
elec_lcow = []
elec_int = []
resin_scenario_l = []
ix_cap_list = []
ix_onm_list = []
ro_cap_list = []
ro_onm_list = []

def save_results(scenario = None, s_value = None, resin_name = None):
    lcow_list.append(value(m.fs.costing.LCOW))
    water_recovery_list.append(value(m.fs.costing.system_recovery))
    scenario_value.append(s_value)
    scenario_name.append(scenario)
    elec_lcow.append(value(m.fs.costing.elec_frac_LCOW))
    elec_int.append(value(m.fs.costing.electricity_intensity))
    if resin_name is None:
        resin_scenario_l.append(0)
        ix_cap_list.append(0)
        ix_onm_list.append(0)    
    else:
        resin_scenario_l.append(resin_name)
        ix_cap_list.append(m.fs.ion_exchange.costing.fixed_cap_inv())
        ix_onm_list.append(m.fs.ion_exchange.costing.annual_op_main_cost())
    
    ro_cap = 0
    ro_onm = 0
    for key in m.fs.pfd_dict.keys():
        if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
            ro_cap = getattr(m.fs, key).costing.fixed_cap_inv() + ro_cap
            ro_onm = getattr(m.fs, key).costing.annual_op_main_cost() + ro_onm
    ro_cap_list.append(ro_cap)
    ro_onm_list.append(ro_onm)

In [19]:
###########################################################    
############################################################
print("-------", "RESET", "-------")
wt.run_water_tap(m = m, objective=False, skip_small = True)
print("LCOW -->", m.fs.costing.LCOW())

# INFLOW
stash_value = []
for key in m.fs.flow_in_dict:   
    stash_value.append(value(getattr(m.fs, key).flow_vol_in[0]))
        
stash_value1 = []
for key in m.fs.flow_in_dict:   
    stash_value1.append(value(getattr(m.fs, key).conc_mass_in[0, "tds"]))
            
stash_value11 = ix.anion_res_capacity[0]()
stash_value22 = ix.cation_res_capacity[0]()
         
tds_removal = []
inflow = []
tds_in = []
capacity = []
    
############ tds_removal 10-50% ############

scenario = "tds_removal 10-90%"
print("-------", scenario, "-------")
ub = 0.9
lb = 0.1
step = (ub - lb) / 10

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()

# TDS REMOVAL        
for i in np.arange(lb, ub + step, step):
    ix.removal_fraction[0, "tds"].fix(i)   
    
    for ii in np.arange(0.9, 2, (2-0.9) / 10):
        q=0
        sensitivity_flow = {}
        for key in m.fs.flow_in_dict:
            getattr(m.fs, key).flow_vol_in[0].fix(stash_value[q] * ii)
            sensitivity_flow[key] = stash_value[q] * ii
            q = q + 1
            
        # TDS IN    
        for iii in np.arange(0.9, 2, (2-0.9) / 10):
            q=0
            for key in m.fs.flow_in_dict:
                getattr(m.fs, key).conc_mass_in[0, "tds"].fix(stash_value1[q] * iii)
                q = q + 1
               
            # CAP    
            for iv in np.arange(0.9, 3, (3-0.9) / 10):
                ix.anion_res_capacity.fix(stash_value11 * iv)
                ix.cation_res_capacity.fix(stash_value22 * iv)
    
                
                wt.run_water_tap(m = m, objective=False, skip_small = True, sensitivity_flow = sensitivity_flow)
                print("LCOW -->", m.fs.costing.LCOW())
                save_results(scenario = scenario, s_value = i, resin_name = ix.resin_name)
                tds_removal.append(i)
                inflow.append(ii)
                capacity.append(iv)
                tds_in.append(iii)


------- RESET -------
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3010934586754306
------- tds_removal 10-90% -------
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2768074239540036
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.27570191838761093
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.27498385223602606
-------------------------------------------------


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2851529686203793
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2848296590837738
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2845683585108509
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2843528687410294
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.30212790894800645
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.30104009800814063
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.300247619187344
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.29964565408411864
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW 


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2746400453812918
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.28234411124095876
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2809960075274358
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2801276848837131
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW -


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2883164654347926
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.28801483270154604
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.28776674414381376
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.2875591828363586
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW 

...


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3049688347118217
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.577917499627385
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.482003881275867
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4257817178053214
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.36639779529988487
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.35182806272203054
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.34043346662274526
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.33133569123889267
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCO


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3470144353750375
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3311550298797395
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3198074069735394
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3113757021809673
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.6478220103282307
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.529979919803214
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4609477913692049
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.4168554337005311
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW -->


WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.367552415683727
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.3539909761180303
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.34317504781501706
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --> 0.8750349881818403
----------------------------------------------------------------------

Degrees of Freedom: 4

WaterTAP3 solution optimal 

----------------------------------------------------------------------
LCOW --

In [20]:
len(tds_in)

11000

In [21]:
sens_df = pd.DataFrame()

sens_df["lcow"] = lcow_list
sens_df["water_recovery"] =  water_recovery_list
sens_df["elec_lcow"] =  elec_lcow
sens_df["elec_int"] =  elec_int
sens_df["scenario_value"] = scenario_value
sens_df["scenario_name"] = scenario_name
sens_df["lcow_difference"] =  sens_df.lcow - value(m.fs.costing.LCOW)
sens_df["water_recovery_difference"] = (sens_df.water_recovery - value(m.fs.costing.system_recovery))
sens_df["resin"] = resin_scenario_l
sens_df["ix_cap"] = ix_cap_list
sens_df["ix_onm"] = ix_onm_list
sens_df["ro_cap"] = ro_cap_list
sens_df["ro_onm"] = ro_onm_list

sens_df["tds_removal"] = ix_cap_list
sens_df["inflow"] = ix_onm_list
sens_df["capacity"] = ro_cap_list
sens_df["tds_in"] = ro_onm_list

sens_df.elec_lcow = sens_df.elec_lcow * 100
sens_df.water_recovery = sens_df.water_recovery * 100

m_scenario = "ps_foa"
sens_df.to_csv("results/case_studies/%s_%s_sensitivity2.csv" % ("KB_FOA", m_scenario))